# Imports and settings

In [1]:
import numpy as np
import scipy
import pandas as pd

from astropy.table import Table
from astropy import units as u
from astropy import constants as const
from astroquery.gaia import Gaia
from astropy import table

from tqdm import tqdm

import stam

import matplotlib.pyplot as plt
from matplotlib import colors
from matplotlib import ticker
%matplotlib tk

plt.style.use('Nature')

from IPython.display import Image

In [2]:
# Nature standards: https://www.nature.com/nature/for-authors/final-submission
figwidth_single = 89/25.4  # [inch]
figwidth_double = 183/25.4  # [inch]

# Read files

In [3]:
# read the candidate table
# sources = Table.read('./data/xmatch.fits')
## i = 33 BH
## i = 40 WD

sources = Table.read('../table_A.fits')

sources['mh'] = [0 if (qf >=8 or np.ma.is_masked(qf) or np.ma.is_masked(feh)) else feh for qf, feh in zip(sources['quality_flags'], sources['Fe_H_est'])]
sources['av_final'] = [acrist if np.ma.is_masked(bayestar) else bayestar for acrist, bayestar in zip(sources['av_acrist'], sources['av_bayestar'])]

In [6]:
sources = Table.read('../table_cut.fits', format='fits')

sources['mh_for_mass_interp'] = [feh_cluster if (qf >=8 or np.ma.is_masked(qf) or np.ma.is_masked(feh)) else feh for qf, feh, feh_cluster in zip(sources['quality_flags'], sources['Fe_H_est'],sources['Fe_H_cluster']) ]
sources['av_for_mass_interp'] = [acrist if np.ma.is_masked(bayestar) else bayestar for acrist, bayestar in zip(sources['av_acrist'], sources['av_bayestar'])]

In [5]:
sources = Table.read('../table_B.fits')

In [7]:
# read the PARSEC models
PARSEC_path = '../data/PARSEC v1.2S/Gaia DR3/'
models = stam.getmodels.read_parsec(path=PARSEC_path)

Taking PARSEC files from ../data/PARSEC v1.2S/Gaia DR3/


In [10]:
# Gaia DR3 clean 100pc for HR diagram plot
gaia = Table.read('../data/other/GaiaDR3_100pc_clean.fits')

# Verify that the isochrone interpolation is OK

In [62]:
# age = 0.16  # [Gyr]
i = 5
# age = np.round(10**sources['log_age_50'][i] * 1e-9,2)
log_age = sources['log_age_50'][i]   # [dex]
log_age_res = 0.025  # [dex]
mh = np.round(np.max([np.min([sources['mh_for_mass_interp'][i], 0.6]), -2]), 1)  # [M/H] round to the nearest available track  # [M/H]
# mh = sources['mh_xgboost'][i]
# age_res = -0.1  # fractional age bin resolution (the minus marks it as a fractional value)
stage_min = 0  # pre-main sequence
stage_max = 3  # red giant branch
mass_min = 0  # [Msun]
mass_max = 8  # [Msun]

In [71]:
filename = './CheckTrackInterpolation'

tracks = stam.gentracks.get_isotrack(models, [log_age, mh], params=("log_age", "mh"), return_table=True,
                                     log_age_res=log_age_res, mass_min=mass_min, mass_max=mass_max,
                                     stage=None, stage_min=stage_min, stage_max=stage_max, sort_by="age", color_filter1="G_BPmag", color_filter2="G_RPmag",
                 mag_filter="Gmag")

color = "bp_rp"
magnitude = "mg"
# color = "B_I"
# magnitude = "V"

x = np.array(tracks["bp_rp"])
y = np.array(tracks["mg"])
z = np.array(tracks["mass"])

xstep=0.05
ystep=0.05

def tracks2grid(tracks, xparam = "bp_rp", yparam = "mg", xstep=0.05, ystep=0.05):

    xmin = np.min(np.around(tracks[xparam], -int(np.round(np.log10(xstep)))))
    xmax = np.max(np.around(tracks[xparam], -int(np.round(np.log10(xstep)))))
    ymin = np.min(np.around(tracks[yparam], -int(np.round(np.log10(ystep)))))
    ymax = np.max(np.around(tracks[yparam], -int(np.round(np.log10(ystep)))))
    x, y = np.meshgrid(np.arange(xmin, xmax, xstep), np.arange(ymin, ymax, ystep))
            
    return x, y, xmin, xmax, ymin, ymax

grid_x, grid_y, xmin, xmax, ymin, ymax = tracks2grid(tracks, xstep=xstep, ystep=ystep)

fun_type = "linear"

interp = scipy.interpolate.RBFInterpolator(np.array([x, y]).T, z, kernel=fun_type)
# interp = scipy.interpolate.LinearNDInterpolator(np.array([x, y]).T, z, fill_value=0)

grid = np.array([grid_x, grid_y])
grid_flat = grid.reshape(2, -1).T
grid_z = interp(grid_flat).reshape(grid_x.shape)
# grid_z = scipy.interpolate.griddata(np.array([x, y]).T, z, grid_flat, method="cubic").reshape(2, -1).T


fig, ax = plt.subplots(figsize=(figwidth_single, figwidth_single), tight_layout=True)
ax.plot(x, y, 'ko', markersize=1)
h = ax.imshow(grid_z, origin="lower", extent=[xmin, xmax, ymin, ymax], cmap='viridis', aspect='auto')


## plot selected source on the diagram

##### without correcting for extinction
bp_rp = sources[color][i]
mg = sources[magnitude][i]
ax.plot(bp_rp, mg, '.r',label='original')
ax.text(bp_rp*1.05, mg*0.95, f"{interp(np.array([[bp_rp, mg]]))[0]:.2f}M$_\odot$", c='r', fontsize=6, va="baseline", ha="left")

#### cluster extinction correction
e_bv = sources['av_for_mass_interp'][i]/3.1
e_bprp, A_G = stam.gaia.get_extinction_in_band(e_bv,mag_filter="G",color_filter1='G_BP',color_filter2='G_RP')
bp_rp = sources[color][i] - e_bprp
mg = sources[magnitude][i] - A_G
ax.plot(bp_rp, mg, '.', c='yellow',label='dereddened')
ax.text(bp_rp*1.05, mg*0.95, f"{interp(np.array([[bp_rp, mg]]))[0]:.2f}M$_\odot$", c='yellow', fontsize=6, va="baseline", ha="left")

plt.colorbar(h, label=r"Mass (M$_\odot$)")
ax.set_xlabel(r"$G_\text{BP}-G_\text{RP}$")
ax.set_ylabel(r"$G$")
ax.set_title(f"{10**log_age*1e-6:.1f} Myr, [M/H]={mh:.2f}; {fun_type} interpolation")
ax.invert_yaxis()
ax.legend()
plt.show()

# fig.savefig(filename + ".png", dpi=300)
# Image(filename + ".png")


# Assign masses, and determine red-excess probability

In [11]:
log_age_res = 0.025  # [dex]
stage_min = 0  # pre-main sequence
stage_max = 3  # red giant branch
mass_min = 0  # [Msun]
mass_max = 8  # [Msun]
reddening_key = "av_for_mass_interp"
n_realizations = 10000
correct_extinction = True

In [12]:
# initialize
sources['m1'] = np.full(len(sources),np.nan)
sources['m1_err'] = np.full(len(sources),np.nan)
sources['red_excess_prob'] = np.full(len(sources),np.nan)

In [ ]:
for i in tqdm(range(len(sources))):
    # get age and metallicity of current source
    log_age = sources['log_age_50'][i]
    mh = np.round(np.max([np.min([sources['mh_for_mass_interp'][i], 0.6]), -2]), 1)  # [M/H] round to the nearest available track
    try:
    #  # assign mass and red-excess probability
          sources['m1'][i], sources['m1_err'][i], sources['red_excess_prob'][i] = stam.run.multirun(sources[i:i+1], vals=[log_age, mh], params=("log_age", "mh"), suffix="", is_save=False,
                                                                                            track_type="isotrack", assign_param="mass", get_excess="red", is_extrapolate=False, rbf_func="linear",
                                                                                              output_type="csv", output_path="./stam_output/", n_realizations=n_realizations, interp_fun="griddata",
                                                                                              models=models, correct_extinction=correct_extinction, reddening_key=reddening_key,
                                                                                              use_reddening_key=True, mass_min=mass_min, mass_max=mass_max,stage=None, stage_min=stage_min, stage_max=stage_max, log_age_res=log_age_res)
    
        # s1, s2, sources['parsec_red_excess_prob'][i] = stam.run.multirun(sources[i:i+1], vals=[log_age, mh], params=("log_age", "mh"), suffix="", is_save=True,
        #                                                                                     track_type="isotrack", assign_param="mass", get_excess="red", is_extrapolate=False, rbf_func="linear",
        #                                                                                     output_type="csv", output_path="./stam_output/", n_realizations=n_realizations, interp_fun="griddata",
        #                                                                                     models=models, correct_extinction=correct_extinction, reddening_key=reddening_key,
        #                                                                                     use_reddening_key=True, mass_min=mass_min, mass_max=mass_max,stage=None, stage_min=stage_min, stage_max=stage_max, log_age_res=log_age_res,
        #                                                                                     color_filter1="B", color_filter2="I", mag_filter="V")
    except:
        print(f"{i}: couldn't assign")

# Comparison of interpolated masses, with NSS masses

In [ ]:
figname = "./img/M1_comparison"

fig, ax = plt.subplots(figsize=(figwidth_single, figwidth_single), tight_layout=True)

# ax.hist(sources['m1']- m1)
ax.errorbar(sources['m1'], sources['parsec_m1'], xerr=np.array([sources['m1'] - sources['m1_lower'], sources['m1_upper']-sources['m1']]), yerr=sources['parsec_m1_parsec_error'], fmt='.k')
ax.plot([0.5, 2.5], [0.5 ,2.5], '--r')
ax.set_xlabel(r'$M_{1,\text{NSS}}$')
ax.set_ylabel(r'$M_{1,\text{PARSEC}}$')
# ax.set_aspect('equal')


# fig.savefig(figname + ".png", dpi=300)
# Image(figname + ".png")

# Plot effect of dereddening on CMD

In [ ]:
figname = "./img/CMD_M1_PARSEC"

fig, ax = plt.subplots(figsize=(figwidth_single, figwidth_single), tight_layout=True)

ax.hist2d(gaia["bp_rp"], gaia["mg"], bins=500, cmap="Greys", norm=colors.PowerNorm(0.5), zorder=0.5)


correct_extinction = True

if correct_extinction:
    e_bv = sources['a_v_50'] / 3.1
    e_bprp, A_G = stam.gaia.get_extinction_in_gaia_band(e_bv)
    bp_rp = sources["bp_rp"] - e_bprp
    mg = sources["mg"] - A_G
else:
    bp_rp = sources["bp_rp"]
    mg = sources["mg"]

h = ax.scatter(bp_rp, mg, s=5, c=m1)
ax.invert_yaxis()

ax.set_xlabel(r"$G_\text{BP} - G_\text{RP}$")
ax.set_ylabel(r"$G_\text{abs}$")

ax.set_xlim([-1, 5.5])
ax.set_ylim([17.6, -2])

plt.colorbar(h, label=r'$M_{1,\text{PARSEC}}$')

plt.show()

# fig.savefig(figname + ".png", dpi=300)
# Image(figname + ".png")

In [ ]:
figname = "./img/CMD_M1_NSS"


fig, ax = plt.subplots(figsize=(figwidth_single, figwidth_single), tight_layout=True)

ax.hist2d(gaia["bp_rp"], gaia["mg"], bins=500, cmap="Greys", norm=colors.PowerNorm(0.5), zorder=0.5)


correct_extinction = False

if correct_extinction:
    e_bv = sources['a_v_50'] / 3.1
    e_bprp, A_G = stam.gaia.get_extinction_in_gaia_band(e_bv)
    bp_rp = sources["bp_rp"] - e_bprp
    mg = sources["mg"] - A_G
else:
    bp_rp = sources["bp_rp"]
    mg = sources["mg"]

h = ax.scatter(bp_rp, mg, s=5, c=sources['m1'])

ax.invert_yaxis()

ax.set_xlabel(r"$G_\text{BP} - G_\text{RP}$")
ax.set_ylabel(r"$G_\text{abs}$")

plt.colorbar(h, label=r'$M_{1,\text{NSS}}$')

ax.set_xlim([-1, 5.5])
ax.set_ylim([17.6, -2])

plt.show()

# fig.savefig(figname + ".png", dpi=300)
# Image(figname + ".png")

In [15]:
from astropy.io import fits
hdu = fits.BinTableHDU(sources)
hdu.writeto('xmatch.fits',overwrite=True)